# Mapping Streamgage HU



In [1]:

import os
import warnings

import folium
import geopandas as gpd
from IPython.display import Image
import pandas as pd


DATA_DIR = os.path.join("..", "..", "data")

In [2]:
ca_fp = os.path.join(DATA_DIR, "NHD", "NHD_H_California_State_Shape", "Shape")
ca_shapefiles = list(filter(lambda s: s.endswith(".shp"), os.listdir(ca_fp)))
wbd_shapefiles = list(filter(lambda s: s.startswith("WBDHU"), ca_shapefiles))

In [3]:
# Read in the shape files for each HU level:
geo_dfs = dict()
for wbd_filename in wbd_shapefiles:
    fp = os.path.join(ca_fp, wbd_filename)
    gdf = gpd.read_file(fp)
    shapefile_name = wbd_filename.split(".")[0]
    gdf["shapefile_name"] = shapefile_name
    geo_dfs[shapefile_name] = gdf

In [4]:
# Combine all Geo dfs into one:
gdfs_list = list(geo_dfs.values())
gdf = gpd.GeoDataFrame(pd.concat(gdfs_list, ignore_index=True), 
                       crs=gdfs_list[0].crs)

In [5]:
# Get our target gage ids:
sg_df = pd.read_csv(os.path.join(DATA_DIR, "streamgage-full.csv"), encoding="utf-8")
gage_ids = sg_df["gage"].unique()

In [6]:
# Get the full metadata for each gage which includes lat-long:
inventory_fp = os.path.join(DATA_DIR, "inventory_clean")
inventory = pd.read_csv(inventory_fp, sep="\t")
target_gages = inventory[inventory["site_no"].isin(gage_ids)]

In [7]:
def get_streamgage_lat_long(gage_id: int):
    """For a streamgage ID, return the lat-long coordinates of its location."""
    s = target_gages.set_index("site_no").loc[gage_id]
    return s["dec_lat_va"], s["dec_long_va"]

In [8]:
get_streamgage_lat_long(11266500)

(37.71687138, -119.6662788)

In [9]:
def get_lat_long_geometry(lat: float, long: float):
    """For a given lat-long coordinate, return the coordinate 
    as a shapely.geometry.point.Point in a geopandas.array.GeometryArray."""
    df = pd.DataFrame({"longitude": [long], "latitude": [lat]})
    geometry = gpd.points_from_xy(df["longitude"], df["latitude"], crs="EPSG:4326")
    return geometry

In [10]:
lat, long = get_streamgage_lat_long(11266500)
get_lat_long_geometry(lat, long)

<GeometryArray>
Length: 1, dtype: geometry

In [11]:
def get_streamgage_overlapping_watersheds(gage_id: int):
    """For a given streamgage ID return all overlapping watershed geometries."""
    lat, long = get_streamgage_lat_long(gage_id)
    point = get_lat_long_geometry(lat, long)[0]
    gdf_subset = gdf[(gdf["geometry"].contains(point))]
    if not len(gdf_subset):
        warnings.warn(f"No overlapping watershed geometries found for gage_id: {gage_id}")
    return gdf_subset

In [12]:
get_streamgage_overlapping_watersheds(11266500)

,tnmid,metasource,sourcedata,sourceorig,sourcefeat,loaddate,referenceg,areaacres,areasqkm,states,...,ObjectID,geometry,shapefile_name,huc8,huc10,huc14,huc16,huc2,huc6,huc4
5137,{8EF929B8-01C5-40B2-B34E-E86B1482B35E},None,None,None,None,2012-06-11,None,8.124436e+05,3287.85,CA,...,73,"POLYGON ((-119.53528 37.90190, -119.53532 37.9...",WBDHU8,18040008,NaN,NaN,NaN,NaN,NaN,NaN
6093,{1AA816C6-23B0-42DF-A6B4-89576562469F},None,None,None,None,2012-10-24,None,1.218170e+05,492.98,CA,...,889,"POLYGON ((-119.53528 37.90190, -119.53532 37.9...",WBDHU10,NaN,1804000803,NaN,NaN,NaN,NaN,NaN
6450,{D924BD6E-507C-4E5B-A86C-DDDDE00752A5},{FE4AD22E-A115-40E4-9EC7-8497E5725E31},None,Natural Resources and Conservation Service and...,None,2022-03-15,None,1.078924e+08,436625.32,"CA,MX,NV,OR",...,4,"MULTIPOLYGON (((-123.01418 37.94333, -123.0148...",WBDHU2,NaN,NaN,NaN,NaN,18,NaN,NaN
6459,{F26FDAD1-B4ED-4D49-9473-4DDE6820E23A},{A0039473-A809-4129-8E12-32F32F5B835C},None,U.S. Geological Survey,None,2022-01-03,None,1.012790e+07,40986.21,CA,...,9,"POLYGON ((-120.54010 38.75126, -120.53834 38.7...",WBDHU6,NaN,NaN,NaN,NaN,NaN,180400,NaN
6482,{419D90DC-A7AA-4C8F-8269-2C3F59883D1F},{A0039473-A809-4129-8E12-32F32F5B835C},None,U.S. Geological Survey,None,2022-01-03,None,1.012790e+07,40986.21,CA,...,8,"POLYGON ((-120.54010 38.75126, -120.53834 38.7...",WBDHU4,NaN,NaN,NaN,NaN,NaN,NaN,1804


In [13]:
def get_streamgage_watershed(gage_id: int, hu: int):
    """For a given streamgage ID and hydrological unit resolution,
    return the watershed geometry for that point."""
    lat, long = get_streamgage_lat_long(gage_id)
    geos = get_streamgage_overlapping_watersheds(gage_id)
    return geos[(geos["shapefile_name"].str.lower().str.endswith(f"hu{hu}"))]

In [14]:
# This is the HU identified for the streamgage by USGS at:
# https://waterdata.usgs.gov/nwis/inventory?agency_code=USGS&site_no=11266500
gdf_subset = get_streamgage_watershed(11266500, 8)
gdf_subset

,tnmid,metasource,sourcedata,sourceorig,sourcefeat,loaddate,referenceg,areaacres,areasqkm,states,...,ObjectID,geometry,shapefile_name,huc8,huc10,huc14,huc16,huc2,huc6,huc4
5137,{8EF929B8-01C5-40B2-B34E-E86B1482B35E},None,None,None,None,2012-06-11,None,812443.61,3287.85,CA,...,73,"POLYGON ((-119.53528 37.90190, -119.53532 37.9...",WBDHU8,18040008,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
def map_streamgage_watershed(gage_id: int, hu: int, zoom_start: int = 5):
    """Map a streamgage and the overlapping watershed at the given HU resolution."""
    lat, lon = get_streamgage_lat_long(gage_id)
    
    my_map = folium.Map(location=[lat, lon], zoom_start=zoom_start, width="80%", height="100%")
    folium.Marker(location=[lat, lon], 
                  radius=10, popup=f"{gage_id} ({lat}, {lon})").add_to(my_map)

    watershed = get_streamgage_watershed(gage_id, hu)
    if not len(watershed):
        warnings.warn(f"\n  No watershed found for HU{hu}")
    for _, r in watershed.iterrows():
        sim_geo = gpd.GeoSeries(r["geometry"]).simplify(tolerance=0.001)
        geo_j = sim_geo.to_json()
        geo_j = folium.GeoJson(data=geo_j,
                               style_function=lambda x: {"fillColor": "orange"})
        label = watershed.iloc[0]["shapefile_name"]
        folium.Popup(label).add_to(geo_j)
        geo_j.add_to(my_map)

    return my_map

In [16]:
map_streamgage_watershed(11266500, 8, 9)